# Including promoted items in recommendations

This notebook will walk you through an example of using promotions in [Amazon Personalize](https://aws.amazon.com/personalize/) to ensure the results for a recommender contain specific items that you want users to see. 

Personalized recommendations increase user engagement with your websites or apps, but often there are additional business rules you want to apply when deciding what items you want to present to your users.  [Amazon Personalize promotions](https://docs.aws.amazon.com/personalize/latest/dg/promoting-items.html) can help when you want a portion of the items presented to users to be from specific categories.  For example, a video on demand app may want to include promotions for new shows or an ecommerce website might want to include sale items in reommendations.  

![promotions-overview.png](images/promotions-overview.png "Diagram showing how promotions changes the result of recommended items")

First, we'll follow the steps to build a Domain dataset group and a recommender that returns product recommendations based on synthetic data generated for a fictitious retail store data set. The goal is to recommend products that are relevant for each particular user.

Then, we'll create a promotions filter to ensure that certain promoted items are present in the recommended items for each user.

Finally, we'll cleanup all of the resources we created so we avoid incurring costs for resources that are no longer being used.  

The estimated time to run through this notebook is about 40 minutes.  

## How to use the Notebook

The code is broken up into cells like the one below. There's a triangular Run button at the top of this page that you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.

Simply follow the instructions below and execute the cells to get started.

## Introduction to Amazon Personalize

[Amazon Personalize](https://aws.amazon.com/personalize/) makes it easy for customers to develop applications with a wide array of personalization use cases, including real time product recommendations and customized direct marketing. Amazon Personalize brings the same machine learning technology used by Amazon.com to everyone for use in their applications – with no machine learning experience required. Amazon Personalize customers pay for what they use, with no minimum fees or upfront commitment. 

You can start using Amazon Personalize with a simple three step process, which only takes a few clicks in the AWS console, or a set of simple API calls. 

First, point Amazon Personalize to user data, catalog data, and activity stream of views, clicks, purchases, etc. in Amazon S3 or upload using a simple API call. 

Second, with a single click in the console or an API call, train a private recommendation model for your data. 

Third, retrieve personalized recommendations for any user by creating a recommender, and using the GetRecommendations API.

If you are not familiar with Amazon Personalize, you can learn more about the service on by looking at [Github Sample Notebooks](https://github.com/aws-samples/amazon-personalize-samples) and [Product Documentation](https://docs.aws.amazon.com/personalize/latest/dg/what-is-personalize.html).

## Imports
Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) which are core data science tools.

In [ ]:
# Get the latest version of botocore to ensure we have the latest features in the SDK
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime3 = boto3.client('personalize-runtime')

Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [ ]:
personalize.list_dataset_groups()

Setup names for items and interactions files to use later

In [ ]:
interactions_file_path = 'cleaned_interactions_training_data.csv'
items_file_path = 'cleaned_item_training_data.csv'

## Specify an S3 Bucket and Data Output Location

Amazon Personalize will need an S3 bucket to act as the source of your data. The code bellow will create a bucket with a unique `bucket_name`.

The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources. 

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("Bucket already exists. Using bucket", bucket_name)

## Download, Prepare, and Upload Training Data

We generated the synthetic data based on the code in the [Retail Demo Store project](https://github.com/aws-samples/retail-demo-store). Follow the link to learn more about the data and potential uses.

First we need to download the data (training data). In this tutorial we'll use the Purchase history from a retail store  dataset. The dataset contains the user_id, item_id, the interactions between customers and items and the time this interaction took place (Timestamp).

### Download and Explore the Interactions Dataset

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/interactions.csv .

The dataset has been successfully downloaded as interactions.csv

Lets learn more about the dataset by viewing its charateristics

In [ ]:
df = pd.read_csv('./interactions.csv')
df

In [ ]:
df.info()

From the cells above, we've learned that our data has has 5 columns, 675004 rows and the headers are: ITEM_ID, USER_ID, EVENT_TYPE, TIMESTAMP and DISCOUNT.

To be compatible with an Amazon Personalize interactions schema, this dataset requires column headings compatible with Amazon Personalize default column names (read about column names [here](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html) )

The ECOMMERCE recommenders require you to provide specific EVENT_TYPE values in order to understand the context of an interaction. Let's look at what event types are currently in our dataset:

In [ ]:
df.EVENT_TYPE.value_counts()

We can see that 'View' and 'Purchase' event are present and we can proceed. 

### Prepare the Interactions Data


### Drop Columns

Some columns in this dataset would not add value to our model and as such need to be dropped from this dataset. Columns such as *discount*.

In [ ]:
test=df.drop(columns=['DISCOUNT'])
df=test
df.sample(10)

In the cell below, we will write our cleaned data to a file named "final_training_data.csv

In [ ]:
df.to_csv(interactions_file_path)

### Download and Explore the Items Dataset

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/items.csv .

The dataset has been successfully downloaded as items.csv

Lets learn more about the dataset by viewing its charateristics

In [ ]:
items_df = pd.read_csv('./items.csv')
items_df

In [ ]:
items_df.info()

Let's explore the kinds of items included in the dataset.

In [ ]:
items_df.CATEGORY_L1.unique()

In [ ]:
items_df.CATEGORY_L2.unique()

### Drop Columns

Some columns in this dataset could add value to our model but are not relevant for this example. For simplicity, we will drop them from this dataset. Columns such as *product_decription*.

In [ ]:
test=items_df.drop(columns=['PRODUCT_DESCRIPTION'])
items_df=test
items_df.sample(10)

Write our cleaned data to a .csv file 

In [ ]:
items_df.to_csv(items_file_path)

## Configure an S3 bucket and an IAM role

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. However, Amazon Personalize will need an S3 bucket to act as the source of your data, as well as IAM roles for accessing that bucket. Let's set all of that up.


## Set the S3 bucket policy
Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

Note: Make sure the role you are using to run the code in this notebook has the necessary permissions to modify the S3 bucket policy.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### Upload Interactions data to S3
Now that our training data is ready for Amazon Personalize,the next step is to upload it to the s3 bucket created earlier

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path
    

### Upload Items data to S3
Now that our training data is ready for Amazon Personalize,the next step is to upload it to the s3 bucket created earlier

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_file_path).upload_file(items_file_path)
items_s3DataPath = "s3://"+bucket_name+"/"+items_file_path

## Create and Wait for Dataset Group
The largest grouping in Personalize is a Dataset Group, this will isolate your data, event trackers, solutions, Recommenders, and campaigns. Grouping things together that share a common collection of data. Feel free to alter the name below if you'd like. 

When you create a Domain dataset group, you choose your domain. The domain you specify determines the default schemas for datasets and the use cases that are available for recommenders. 

You can find more information about creating a Domain dataset group in [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/create-domain-dataset-group.html).

### Create Dataset Group

In [ ]:
response = personalize.create_dataset_group(
    name='personalize_ecomemerce_ds_group',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Wait for Dataset Group to Have ACTIVE Status
Before we can use the Dataset Group in any items below it must be active, execute the cell below and wait for it to show active.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Create Interactions Schema
A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
            
        }
        
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Create Items Schema
A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "PRICE",
            "type": "float"
        },
        {
            "name": "CATEGORY_L1",
            "type": ["string"],
            "categorical": True
        },
        {
            "name": "CATEGORY_L2",
            "type": ["string"],
            "categorical": True
            
        },
        {
            "name": "GENDER",
            "type": ["string"],
            "categorical": True
            
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-item_group",
    domain = "ECOMMERCE",
    schema = json.dumps(items_schema)
)

items_schema_arn = create_schema_response['schemaArn']

print(json.dumps(create_schema_response, indent=2))

## Create Datasets
After the group, the next thing to create is the datasets where your data will be uploaded to in Amazon Personalize.

### Create Interactions Dataset

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Create Items Dataset

In [ ]:
dataset_type = "ITEMS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Create Personalize Role
Also Amazon Personalize needs the ability to assume Roles in AWS in order to have the permissions to execute certain tasks, the lines below grant that.

Note: Make sure the role you are using to run the code in this notebook has the necessary permissions to create a role.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Import the data
Earlier you created the DatasetGroup and Dataset to house your information, now you will execute an import job that will load the data from S3 into Amazon Personalize for usage building your model.
### Create Interactions Dataset Import Job

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### Create Items Dataset Import Job

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_items_import",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_file_path)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### Wait for Dataset Import Jobs to Have ACTIVE Status
It can take a while before the import jobs complete, please wait until you see that they are active below.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("ItemsDatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("InteractionsDatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Choose a recommender use case

Each domain has different use cases. When you create a recommender you create it for a specific use case, and each use case has different requirements for getting recommendations.


In [ ]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain. 
display (available_recipes['recipes'])

We are going to create a recommender of the type "Recommended For You". This type of recommender offers personalized recommendations for items based on a user that you specify. With this use case, Amazon Personalize automatically filters items the user purchased based on the userId that you specify and `Purchase` events.

[More use cases per domain](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

We wait until the recomenders have finished creating and have status `ACTIVE`. We check periodically on the status of the recommender

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommended_for_you_arn
    )
    status = version_response["recommender"]["status"]
    print(status)

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommended_for_you_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommended_for_you_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Recommended for you" Recommender build is still in progress')
        
    time.sleep(60)

## Getting recommendations with a recommender
Now that the recommender has been trained, lets have a look at the recommendations we can get for our users!

In [ ]:
# reading the original data in order to have a dataframe that has both item_ids 
# and the corresponding titles to make our recommendations easier to read.
items_df = pd.read_csv('./items.csv')
items_df.sample(10)

In [ ]:
def get_item_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item description.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Error obtaining item description"

In [ ]:
def get_category_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item category.
    
    A really broad try/except clause was added in case anything goes wrong.
    """
    
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['CATEGORY_L2'].values[0]
    except:
        print (item_id)
        return "Error obtaining item category"
    

Let us get some  recommendations from the recommender returning "Recommended for you":

In [ ]:
# First pick a user
test_user_id = "777" 

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns = ["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows =20
display(user_recommendations_df)

## Using promotions with a recommender
Now, lets create a promotion to ensure that recommendations for users contain specific items we want to promote.  In this example our ecommerce store is promoting items for the upcoming Halloween holiday.  So, we want to make sure that users see halloween items.

For more information on how to use promotions, please refer to [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/promoting-items.html). 

### Create a filter to use with the promotion

First we need to create a filter that will define what items will be promoted. We will use a dynamic [filter](https://docs.aws.amazon.com/personalize/latest/dg/filter-expressions.html) that takes in a value at inference time. 

This filter will include only items that have the specified value for *Items.CATEGORY_L2*.

To get the same results we could also use a static filter of the form:

```
'INCLUDE ItemID WHERE Items.CATEGORY_L2 IN ("halloween")'
```

however, using the dynamic filter gives the customer more flexibility if they later want to promote items in a different category without having to create a different filter.

In [ ]:
create_filter_response = personalize.create_filter(
    name = 'category_filter',
    datasetGroupArn = dataset_group_arn,
    filterExpression = 'INCLUDE ItemID WHERE Items.CATEGORY_L2 IN ($CATEGORY)'
) 
filter_arn = create_filter_response["filterArn"]
print("Filter ARN: " + filter_arn)

Wait for the filter we created to have status "Active".

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
    
while time.time() < max_time:
    version_response = personalize.describe_filter(
        filterArn = filter_arn
    )
    status = version_response["filter"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(filter_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(filter_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The Filter build is still in progress')
        
    time.sleep(30)

### Get recommendations for our test user using the filter 

Now that the filter has been created, we can get recommendations for a user using the filter. Let's check our existing test user.

In [ ]:
print(test_user_id)

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20,
    filterArn = filter_arn,
    filterValues={"CATEGORY" : "\"halloween\""}
)
user_recommendations_df =[]

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns = ["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows =20
display(user_recommendations_df)

As you can see from the results, all returned items have the category level 2 "halloween".

### Get recommendations using the same filter with promotions

We want to make personalized recommendations for each user, but instead of having all items be of a certain category, we want to still include some items for the Halloween promotion.  Let's use _promotions_ to make it happen.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20,
    promotions = [{
        "name" : "halloween_promotion",
        "percentPromotedItems" : 20,
        "filterArn": filter_arn,
        "filterValues": {
            "CATEGORY" : "\"halloween\""
        }
    }]
)


# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns = ["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows =20
display(user_recommendations_df)

### Combine filters and promotions

You can use a filter combined with promotions. The filter in the the top-level parameter block applies to only to the non-promoted items. The filter to select the promoted items is specified in the `promotions` parameter block. The following example uses the same dynamic filter we have been using twice. The first filter applies to non-promoted items, selecting items of the categry level 2 "decorative", and the second filter applies to the promotion, promoting items of the category level 2 "halloween".

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn=recommended_for_you_arn,
    userId=test_user_id,
    numResults=20,
    filterArn=filter_arn,
    filterValues={
        "CATEGORY": "\"decorative\""
    },
    promotions=[{
        "name": "halloween_promotion",
        "percentPromotedItems": 20,
        "filterArn": filter_arn,
        "filterValues": {
            "CATEGORY": "\"halloween\""
        }
    }]
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(
        get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns=["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows = 20
display(user_recommendations_df)

## Review
Using the codes above you have successfully trained a deep learning model to generate item recommendations based on prior user behavior. You have created a recommenders for a foundational use case and you have used filters and [promotions](https://docs.aws.amazon.com/personalize/latest/dg/promoting-items.html) to apply additional business rules for items that should be presented to users.

Going forward, you can adapt this code to create other recommenders.

If you are done with this sample, make sure to follow the steps in the next section to cleanup the resources created in this notebook.

## Cleanup Resources 
This section contains instructions on how to clean up the resources created in this notebook

### Save resource information for cleanup:
There are a few values you will need for the next notebook, execute the cell below to store them so they can be used in the `Clean_Up_Resources.ipynb` notebook.

This will overwite any data stored for those variables and set them to the values specified in this notebook. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

### Run the cleanup notebook

Continue to [Clean_Up_Resources.ipynb](Clean_Up_Resources.ipynb) clean up resources.